# Inicialização

Carregamos a seguir as bibliotecas utilizadas.

In [1]:
# Import de bibliotecas utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pyxlsb import open_workbook as open_xlsb

%matplotlib inline

data_xlsb = '../data/ambev.xlsb'

In [2]:
# Executar se pyxlsb não tiver instalado
#!pip install pyxlsb
#from pyxlsb import open_workbook as open_xlsb

# Leitura/Limpeza dos dados

Incluímos aqui nessa seção os passos de tratamento do conjunto de dados realizados no notebook de exploração `AmBev_Data_Wrangling`. Algumas explicações são omitidas aqui, de modo a manter o notebook mais enxuto. Para mais detalhes sobre os passos realizados, favor consultar `AmBev_Data_Wrangling.ipynb`.

Definimos funções auxiliares para realizar a limpeza dos dados.

In [3]:
import unidecode

# Função para substituir espaços por _ (utilizado nos nomes de colunas)
def str_replace_spaces(input_str):
    if input_str == None: return None
    else: return input_str.replace(' ','_')

# Função para 'sanitizar' strings, substituindo erros inseridos por encoding
def str_sanitize(input_str):
    if not (isinstance(input_str,str)):
        return input_str
    
    output_str = input_str.replace('Á¡', 'á')
    
    return unidecode.unidecode(output_str)

# Função que recebe valores de uma linha 'suja' (com valores de Regra Alcance Parcial em mais de uma coluna) 
# e retorna uma linha 'limpa'.
def list_sanitize(input_list):
    output_list = []
    read_val = False
    
    for i, a in enumerate(input_list):
        if i == 21: 
            val = a
            if val != None: val = str(val)
        elif i == 22:
            if a != None \
            and not re.search('Sim',a) \
            and not re.search('Não',a):
                read_val = True
                val += " " + a
            else:
                output_list.append(val)
                output_list.append(a)
        
        else:
            if read_val:
                if a == None:
                    read_val = False
                    output_list.append(val)
                    output_list.append(a)
                    
                if re.search('\"$',str(a)):
                    read_val = False
                    val += " " + a
                    output_list.append(val)
                    
                    
            else:
                output_list.append(a)
                
    return output_list

Definidas as funções auxiliares, vamos realizar a leitura dos dados. Note que: valores ausentes são preenchidos com `nan`, para evitar valores `None` no dataframe; as últimas colunas (vazias) são removidas; e `Mes_Referencia` é tratado como `datetime`, no formato Mês + Ano.

In [21]:
df = []

with open_xlsb(data_xlsb) as wb:
    for sheetname in wb.sheets:
        with wb.get_sheet(sheetname) as sheet:  
            for i,row in enumerate(sheet.rows()):
                row_li = [item.v for item in row]
                if (i>0): row_li = list_sanitize(row_li)
                elif (i==0): row_li = [str_sanitize(str_replace_spaces(item)) for item in row_li]
                df.append(row_li)
                
df = pd.DataFrame(df[1:], columns=df[0])
df.drop(df.columns[-5:-1],axis=1,inplace=True)
df['Mes_Referencia'] = pd.to_datetime(df['Mes_Referencia'], format='%m%Y.0')

Vejamos como ficou o dataframe.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270633 entries, 0 to 270632
Data columns (total 33 columns):
Mes_Referencia           270633 non-null datetime64[ns]
Pais                     270633 non-null object
Mundo                    270501 non-null object
Regional/Area            270633 non-null float64
Unidade                  270633 non-null float64
Grupo_Cargo              230099 non-null object
Cargo                    262118 non-null object
Grade                    262118 non-null float64
Banda                    262118 non-null object
Area                     262116 non-null object
Nome_Funcionario         270633 non-null float64
Nome_Gestor              270633 non-null float64
Codigo_KPI               230229 non-null object
Diretoria                230229 non-null object
Areas_da_Diretoria       230229 non-null object
Funcao                   230229 non-null object
Tipo_da_Meta             270633 non-null object
Categoria_KPI            267176 non-null object
Nome_KPI    

Criamos um dicionário para substituir erros gerais na codificação, e definimos uma função para realizar a limpeza adicional.

In [23]:
replace_dict = {'Áµ':'õ',
                'Ã`\x01': 'Ê',
                'Ã\x8d': 'Í',
                'Ã! ':'Ç',
                'Ã\x92\x01O': 'ÃO',
                'Á\x1d ': 'Ô',
                'Á\x1c': 'Ó', 
                'Á\x8d': 'Í',
                'Â\xa0': ' ',
                '\x81':'',
                'â¬ \x1c ': '-', 
                'Á¡':'á',
                'Ã©': 'é',
                'Ã\xad': 'í',
                'Ã\x1c': 'Ó',
                'Ã£': 'ã',
                'ARMAZÁ0 M': 'ARMAZÉM',
                'OPERAçÁ" ES': 'OPERAçÕES',
                'Logisitca': 'Logistica',
                'Sumistros': 'Suministros',
                'GREG': 'GER'
               }

def str_clean(x):
    if x == None or not isinstance(x,str): return x
    for item in replace_dict:
        if item in x:
            x = x.replace(item,replace_dict[item])
            
    return unidecode.unidecode(x).strip()

Aplicamos a função aos campos que têm string. Além disso, usamos a função `title` para algumas categorias.

In [26]:
for col in df.columns:
    if (df[col].dtype=='O'):
        df[col] = df[col].apply(lambda x : str_clean(x))

cols = ['Grupo_Cargo', 'Area', 'Funcao', 'Categoria_KPI']
for col in cols:
    df[col] = df[col].apply(lambda x : x.title() if x != None else x)

Grupo_Cargo
Area
Funcao
Categoria_KPI


Para refinar a limpeza, e remover inconsistências específicas a cada feature, aplicamos a função abaixo.

In [27]:
replace_dict_cat = {
    'Area': [
        ['E-Commerce', 'Ecommerce'],
        ['Projeto', 'Projetos']
    ],
    
    'Funcao': [
        ['Gerente Financeiro', 'Ger Financeiro']
    ],
    
    'Categoria_KPI': [
        ['Imagen De Mnarca', 'Imagen De Marca'],
        ['Machine Kpis', 'Machine Kpi'],
        ['Cash Flow','Cashflow'],
        ['Project Kpi', 'Project'],
        ['Le Accurancy', 'Le Accuracy'],
        ['Legal Compliance', 'Compliance'],
        ['Gestio N Recursos Marcas', 'Gestion Recursos Marcas']
        
    ],
    
    
    'Prazo': [
        ['monthly', 'Monthly'],
        ['Monthly basis', 'Monthly'],
        ['monthly basis', 'Monthly']
    ]
}

def str_refinement(df):
    for cat in replace_dict_cat:
        for [str_from,str_to] in replace_dict_cat[cat]:
            df[cat].replace(str_from,str_to,inplace=True)
            
str_refinement(df)

Agrupamos então strings similares dentro de uma mesma feature.

In [28]:
from difflib import SequenceMatcher

# Função de similaridade
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()    

# Função para agrupar as diferentes versões de strings em uma coluna
def str_similarity_replacement(df,col):
    replacement_dict = {}
    
    enum = list(enumerate(df[col].unique()))
    for i, a in enum:
        for j,b in enum:
            if (j > i) and (isinstance(a,str)) and (isinstance(b,str)) and (similar(a,b) > 0.7):
                if b not in replacement_dict: replacement_dict[b] = a
                
    for b in replacement_dict:
        a = replacement_dict[b]
        while (a in replacement_dict):
            a = replacement_dict[a]
        replacement_dict[b] = a
        
    for b in replacement_dict:
        df[col].replace(b, replacement_dict[b],inplace=True)
        
        
str_similarity_replacement(df,'Categoria_KPI')

O trabalho que não pôde ser realizado automaticamente pelo agrupamento é então realizado manualmente.

In [34]:
def sanitize_cat_kpi(df):
    col = 'Categoria_KPI'
    
    df[col] = df[col].apply(lambda x: re.sub(r'\s*/\s*', r'/', x).strip() if x != None else x)
    
    replace_dict = {
        'Continuous Improvement': 'Melhoria Continua',
        'People & Management': 'Gente E Gestao',
        'Imagem Da Marca (Brandhealth)': 'Imagem Da Marca',
        'Brand Health' : 'Imagem Da Marca',
        'Innovations/Innovations Project': 'Inovacoes/Projeto De Inovacoes',
        'Quality': 'Qualidade',
        'Imagen De Marca': 'Imagem Da Marca',
        'Rentabilidade/Custos (Maco, Vic, Vlc, Ebitda, Eficiencia E Produtividade)': 'Rentabilidade/Custos',
        'Rentabilidad/Costos': 'Rentabilidade/Custos',
        'Better World' : 'Mundo Melhor',
        'Innovaciones': 'Inovacoes/Projeto De Inovacoes',
        'Profit/Costs (Maco,Vic, Vlc, Ebitda..)': 'Rentabilidade/Custos',
        'Long-Term Projects/Initiatives/Strategy': 'Projetos/Iniciativas De Longo Prazo/Estrategia',
        'Volumen': 'Market Share/Volume',
        'Ebitda': 'Rentabilidade/Custos',
        'Productivity': 'Rentabilidade/Custos',
        'Cost': 'Rentabilidade/Custos',
        'Gestion': 'Gente E Gestao',
        'Sinergias Caribe': 'Sinergias',
        'Synergies': 'Sinergias',
        'People': 'Gente E Gestao',
        'Reputacion': 'Imagem Da Marca',
        'Project': 'Projetos/Iniciativas De Longo Prazo/Estrategia',
        'Safety': 'Seguranca',
        'Seguridad': 'Seguranca',
        'Obz': 'OBZ',
        'Zbb': 'OBZ',
        'Obz Incentivos': 'OBZ',
        'Reporte De Carga Tendencias Obz': 'OBZ',
        'Vpo Sustentable - Management And People Pilar': 'Gente E Gestao',
        'Dpo': 'DPO',
        'Machine Kpi': 'Machine KPI',
        'Rojo+Negro Equipo Frio': 'Equipo Frio',
        'Reporte Equipo Frio Rojo+Negro': 'Equipo Frio'
    }
    
    for a in replace_dict:
        df[col].replace(a, replace_dict[a], inplace=True)
    
    
    
    
sanitize_cat_kpi(df)

In [41]:
df['Status_Meta'].unique()

array(['Monitoramento Aprovado', nan], dtype=object)

In [40]:
df.fillna(value=pd.np.nan, inplace=True)

In [38]:
df.replace('None', np.nan, inplace=True)

In [42]:
df.head()

,Mes_Referencia,Pais,Mundo,Regional/Area,Unidade,Grupo_Cargo,Cargo,Grade,Banda,Area,...,%_Ating_Mes,%_Pontos_Mes,%_Acum_Mes,%_Ating_Acumulado,%_Pontos_Acumulado,%_Acum_Acumulado,%_Ating_Fim_Exer,%_Pontos_Fim_Exer,%_Acum_Fim_Exer,Status_Meta
0,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,25.0,Monitoramento Aprovado
1,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,80.0,60.0,12.0,80.0,60.0,12.0,100.0,100.0,20.0,Monitoramento Aprovado
3,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,100.0,100.0,15.0,90.0,80.0,12.0,100.0,100.0,15.0,Monitoramento Aprovado
4,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


E aqui encerramos a etapa de Limpeza/Manipulação dos dados.

# Exploração dos dados

## Valores nulos de KPI

Vamos analisar as entradas que não possuem valores na feature `Categoria_KPI`.

In [138]:
for col in df.columns:
    print(col)
    print(df[df['Categoria_KPI'].isna()][col].unique())

Mes_Referencia
['2017-06-01T00:00:00.000000000' '2017-07-01T00:00:00.000000000'
 '2017-08-01T00:00:00.000000000' '2017-09-01T00:00:00.000000000'
 '2017-10-01T00:00:00.000000000' '2017-11-01T00:00:00.000000000'
 '2017-12-01T00:00:00.000000000']
Pais
['Barbados' 'Cuba' 'Dominica' 'Guatemala' 'Panama' 'Rep. Dominicana']
Mundo
['Supply' 'Sales' 'Finance' 'Integracion' 'Marketing' 'People' 'Logistica'
 'Sales - Operaciones' 'Sales - Ventas' 'Sales - Finanzas'
 'Sales - Gente y Gestion' 'Sales - Marketing' 'Caribe Ex' 'CE'
 'HILA - Proceso de Ventas' 'HILA - Suministros'
 'Sales - Cadena de Abastecimiento & TI' 'Sales - International'
 'Sales - Legal' 'Sales - Proyectos y TI' 'Sales - Suministros'
 'Sales - Trade Marketing' 'Supply - Planta CND' 'Asuntos Corporativos']
Regional/Area
[ 4.]
Unidade
[  16.   17.  257.  279.  264.  258.  305.  306.  315.  328.  329.   47.
   50.   56.   69.  113.  147.  159.  160.  161.  165.  265.  290.  292.
  312.  325.  316.]
Grupo_Cargo
[nan]
Cargo
[nan 'GE

In [149]:
for a in sorted(df['Regional/Area'].unique()):
    print(str(a) + ": " + str(100*len(df[df['Categoria_KPI'].isna() & (df['Regional/Area']==a)])/len(df[(df['Regional/Area']==a)])))

0.0: 0.0
1.0: 0.0
2.0: 0.0
3.0: 0.0
4.0: 8.556083556083555
5.0: 0.0
6.0: 0.0
7.0: 0.0
8.0: 0.0
9.0: 0.0
10.0: 0.0
11.0: 0.0
12.0: 0.0
13.0: 0.0
14.0: 0.0
15.0: 0.0
16.0: 0.0
17.0: 0.0
18.0: 0.0
19.0: 0.0
20.0: 0.0
21.0: 0.0
22.0: 0.0


In [48]:
cols = ['Peso_KPI']
for col in df.columns:
    if re.search('^\%_',col):
        cols.append(col)
        
print(cols)

['Peso_KPI', '%_Ating_Mes', '%_Pontos_Mes', '%_Acum_Mes', '%_Ating_Acumulado', '%_Pontos_Acumulado', '%_Acum_Acumulado', '%_Ating_Fim_Exer', '%_Pontos_Fim_Exer', '%_Acum_Fim_Exer']


In [74]:
df.groupby(['Nome_Funcionario'])['Mundo'].nunique()

Nome_Funcionario
0.0       1
1.0       1
2.0       1
3.0       2
4.0       1
5.0       1
6.0       1
7.0       1
8.0       1
9.0       1
10.0      1
11.0      1
12.0      1
13.0      1
14.0      1
15.0      1
16.0      1
17.0      1
18.0      1
19.0      1
20.0      1
21.0      1
22.0      1
23.0      1
24.0      1
25.0      1
26.0      1
27.0      1
28.0      1
29.0      1
         ..
7829.0    1
7830.0    1
7831.0    1
7832.0    1
7833.0    1
7834.0    1
7835.0    1
7836.0    1
7837.0    1
7838.0    1
7839.0    1
7840.0    1
7841.0    1
7842.0    1
7843.0    1
7844.0    1
7845.0    1
7846.0    1
7847.0    1
7848.0    1
7849.0    1
7850.0    1
7851.0    1
7852.0    1
7853.0    1
7854.0    1
7855.0    1
7856.0    1
7857.0    1
7858.0    1
Name: Mundo, Length: 7858, dtype: int64

In [63]:
df[df['Nome_Funcionario']==0].groupby(['Mes_Referencia','Categoria_KPI']).count()

Pais  Mundo  \
Mes_Referencia Categoria_KPI                                                 
2017-03-01     Melhoria Continua                                  3      3   
               Rentabilidade/Custos                               2      2   
2017-04-01     Melhoria Continua                                  3      3   
               Rentabilidade/Custos                               2      2   
2017-05-01     Melhoria Continua                                  3      3   
               Rentabilidade/Custos                               2      2   
2017-07-01     Melhoria Continua                                  3      3   
               Rentabilidade/Custos                               2      2   
2017-08-01     Melhoria Continua                                  3      3   
               Rentabilidade/Custos                               2      2   
2017-09-01     Melhoria Continua                                  2      2   
               Projetos/Iniciativas De Longo Prazo/Estrategia     1      1   
               Rentabilidade/Custos                               2      2   
2017-10-01     Melhoria Continua                                  2      2   
               Projetos/Iniciativas De Longo Prazo/Estrategia     1      1   
               Rentabilidade/Custos                               2      2   
2017-11-01     Melhoria Continua                                  2      2   
               Projetos/Iniciativas De Longo Prazo/Estrategia     1      1   
               Rentabilidade/Custos                               2      2   

                                                               Regional/Area  \
Mes_Referencia Categoria_KPI                                                   
2017-03-01     Melhoria Continua                                           3   
               Rentabilidade/Custos                                        2   
2017-04-01     Melhoria Continua                                           3   
               Rentabilidade/Custos                                        2   
2017-05-01     Melhoria Continua                                           3   
               Rentabilidade/Custos                                        2   
2017-07-01     Melhoria Continua                                           3   
               Rentabilidade/Custos                                        2   
2017-08-01     Melhoria Continua                                           3   
               Rentabilidade/Custos                                        2   
2017-09-01     Melhoria Continua                                           2   
               Projetos/Iniciativas De Longo Prazo/Estrategia              1   
               Rentabilidade/Custos                                        2   
2017-10-01     Melhoria Continua                                           2   
               Projetos/Iniciativas De Longo Prazo/Estrategia              1   
               Rentabilidade/Custos                                        2   
2017-11-01     Melhoria Continua                                           2   
               Projetos/Iniciativas De Longo Prazo/Estrategia              1   
               Rentabilidade/Custos                                        2   

                                                               Unidade  \
Mes_Referencia Categoria_KPI                                             
2017-03-01     Melhoria Continua                                     3   
               Rentabilidade/Custos                                  2   
2017-04-01     Melhoria Continua                                     3   
               Rentabilidade/Custos                                  2   
2017-05-01     Melhoria Continua                                     3   
               Rentabilidade/Custos                                  2   
2017-07-01     Melhoria Continua                                     3   
               Rentabilidade/Custos                                  2   
201

Acum = Pontos * Peso_KPI/100

In [55]:
df[~df['%_Pontos_Mes'].isna()].shape

(106285, 33)

In [53]:
df[df['%_Pontos_Mes']*df['Peso_KPI']/100 == df['%_Acum_Mes']].shape

(106285, 33)

In [58]:
df[df['%_Pontos_Mes'] <= df['%_Ating_Mes']].shape

(106285, 33)

In [82]:
df[df['%_Pontos_Mes'].isna()].shape

(164348, 33)

In [80]:
for col in df.columns:
    print(col)
    print(df[df['%_Pontos_Mes'].isna()][col].unique())

Mes_Referencia
['2017-03-01T00:00:00.000000000' '2017-04-01T00:00:00.000000000'
 '2017-05-01T00:00:00.000000000' '2017-06-01T00:00:00.000000000'
 '2017-07-01T00:00:00.000000000' '2017-08-01T00:00:00.000000000'
 '2017-09-01T00:00:00.000000000' '2017-10-01T00:00:00.000000000'
 '2017-11-01T00:00:00.000000000' '2017-12-01T00:00:00.000000000']
Pais
['Brasil' 'Barbados' 'Cuba' 'Dominica' 'Guatemala' 'Panama'
 'Rep. Dominicana' 'Saint Vincent']
Mundo
['Sales' 'Supply' 'CE' 'DG' nan 'Logistica' 'Finance' 'People'
 'Asuntos Corporativos' 'Compras' 'Integracion' 'Marketing' 'Solutions'
 'HILA - Proceso de Ventas' 'Sales - Finanzas' 'Sales - Gente y Gestion'
 'Sales - Operaciones' 'Sales - Trade Marketing' 'Sales - Ventas'
 'Sales - Cadena de Abastecimiento & TI' 'Sales - Marketing' 'Caribe Ex'
 'HILA - Suministros' 'Sales - International' 'Sales - Legal'
 'Sales - Proyectos y TI' 'Sales - Suministros' 'Supply - Planta CND'
 'Supply - Planta HN' 'Comercial' 'Supply - Planta Habana']
Regional/Area

[ 12.  11.  13.  14.  15.   9.  10.   8.   7.  nan   6.  16.  17.   3.   2.
   1.   5.   4.]
Banda
['VI-B' 'VI-C' 'VI-A' 'V-B' 'V-A' 'VII-B' 'VII-A' 'VII-C' 'VIII' nan 'IV-B'
 'IV-A' 'VII-c' 'VI-b' 'VII-b' 'VI-a' 'VII-a' 'VI-c' 'V-a' 'V-b' 'IV-b'
 'Blue Collars' 'VII' 'VI']
Area
['Gerencia Cd' 'Producao' 'Apf' 'Ceng Campo' 'Vendas Brasil' 'Packaging'
 'Distribuicao Rota' 'Manutencao' 'Qualidade Assegurada' 'Gente Gestao'
 'Dir Regional' 'Logistica' 'Processo Cerveja' 'Utilidades' 'Vendas Rota'
 'Processo Refri' 'Ceng' 'Comercial As' 'Gerencia Cd - As' 'Do Bem'
 'Meio Ambiente' 'Gerencia Fabril' 'Adc' 'Financeiro' 'Juridico'
 'Innovation And Analytics' 'Adv' 'Comercial Rota' 'Inativas'
 'Gente Gestao As' 'Capex' 'Apr' 'Cat' 'Rel. Corporativa' 'Cdt'
 'Operacoes Diretoria' 'Suprimentos' 'Design And Delivery' 'Ppm Vendas'
 'Controle' 'Adl' 'Gente Gestao Diretoria' 'Vendas As'
 'Value Creation And Ppm' 'Distribuicao As' 'Ti' 'Noc Transformation' 'Csu'
 'Equipe Refri' 'Specialties' 'Bblend' 

In [91]:
df[df['%_Pontos_Mes'].isna()
  & df['%_Pontos_Acumulado'].isna()
  & df['%_Pontos_Fim_Exer'].isna()].head(10)

,Mes_Referencia,Pais,Mundo,Regional/Area,Unidade,Grupo_Cargo,Cargo,Grade,Banda,Area,...,%_Ating_Mes,%_Pontos_Mes,%_Acum_Mes,%_Ating_Acumulado,%_Pontos_Acumulado,%_Acum_Acumulado,%_Ating_Fim_Exer,%_Pontos_Fim_Exer,%_Acum_Fim_Exer,Status_Meta
1,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017-03-01,Brasil,Supply,20.0,14.0,Supervisor Fabril,SUPERVISOR SENIOR FABRIL,11.0,VI-C,Producao,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017-03-01,Brasil,Supply,20.0,14.0,Supervisor Fabril,SUPERVISOR SENIOR FABRIL,11.0,VI-C,Producao,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2017-03-01,Brasil,CE,3.0,5.0,Especialista,ESPECIALISTA II,13.0,VI-A,Apf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2017-03-01,Brasil,CE,3.0,5.0,Especialista,ESPECIALISTA II,13.0,VI-A,Apf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,2017-03-01,Brasil,CE,3.0,5.0,Especialista,ESPECIALISTA II,13.0,VI-A,Apf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2017-03-01,Brasil,Supply,2.0,195.0,Ger Engenharia,GER ENGENHARIA III,14.0,V-B,Ceng Campo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2017-03-01,Brasil,Supply,2.0,195.0,Ger Engenharia,GER ENGENHARIA III,14.0,V-B,Ceng Campo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2017-03-01,Brasil,Supply,2.0,195.0,Ger Engenharia,GER ENGENHARIA III,14.0,V-B,Ceng Campo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df.groupby('Nome_Funcionario').nunique()

,Mes_Referencia,Pais,Mundo,Regional/Area,Unidade,Grupo_Cargo,Cargo,Grade,Banda,Area,...,%_Ating_Mes,%_Pontos_Mes,%_Acum_Mes,%_Ating_Acumulado,%_Pontos_Acumulado,%_Acum_Acumulado,%_Ating_Fim_Exer,%_Pontos_Fim_Exer,%_Acum_Fim_Exer,Status_Meta
Nome_Funcionario,,,,,,,,,,,,,,,,,,,,,
0.0,8,1,1,1,2,2,2,2,2,2,...,4,4,4,4,4,4,2,2,2,1
1.0,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,1,1,1,1
2.0,9,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,1,1,1,1
3.0,6,1,2,2,2,2,2,1,1,2,...,2,2,2,2,2,2,2,2,3,1
4.0,9,1,1,2,1,1,1,1,1,1,...,2,2,3,1,1,2,1,1,2,1
5.0,7,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
6.0,7,1,1,1,1,1,1,1,1,1,...,3,3,5,4,4,5,3,3,6,1
7.0,8,1,1,2,1,1,1,1,1,1,...,2,2,2,1,1,1,1,1,1,1
8.0,9,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,1,1,2,1


In [118]:
df[~df['%_Pontos_Mes'].isna()
  & ~df['%_Pontos_Acumulado'].isna()
  & ~df['%_Pontos_Fim_Exer'].isna()].head(10)

,Mes_Referencia,Pais,Mundo,Regional/Area,Unidade,Grupo_Cargo,Cargo,Grade,Banda,Area,...,%_Ating_Mes,%_Pontos_Mes,%_Acum_Mes,%_Ating_Acumulado,%_Pontos_Acumulado,%_Acum_Acumulado,%_Ating_Fim_Exer,%_Pontos_Fim_Exer,%_Acum_Fim_Exer,Status_Meta
0,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,25.0,Monitoramento Aprovado
2,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,80.0,60.0,12.0,80.0,60.0,12.0,100.0,100.0,20.0,Monitoramento Aprovado
3,2017-03-01,Brasil,Sales,10.0,78.0,Ger De Vendas Revenda,GERENTE DE VENDAS REVENDA II,12.0,VI-B,Gerencia Cd,...,100.0,100.0,15.0,90.0,80.0,12.0,100.0,100.0,15.0,Monitoramento Aprovado
6,2017-03-01,Brasil,Supply,20.0,14.0,Supervisor Fabril,SUPERVISOR SENIOR FABRIL,11.0,VI-C,Producao,...,100.0,100.0,20.0,100.0,100.0,20.0,100.0,100.0,20.0,Monitoramento Aprovado
8,2017-03-01,Brasil,Supply,20.0,14.0,Supervisor Fabril,SUPERVISOR SENIOR FABRIL,11.0,VI-C,Producao,...,100.0,100.0,20.0,100.0,100.0,20.0,100.0,100.0,20.0,Monitoramento Aprovado
9,2017-03-01,Brasil,Supply,20.0,14.0,Supervisor Fabril,SUPERVISOR SENIOR FABRIL,11.0,VI-C,Producao,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,25.0,Monitoramento Aprovado
10,2017-03-01,Brasil,CE,3.0,5.0,Especialista,ESPECIALISTA II,13.0,VI-A,Apf,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,25.0,Monitoramento Aprovado
11,2017-03-01,Brasil,CE,3.0,5.0,Especialista,ESPECIALISTA II,13.0,VI-A,Apf,...,100.0,100.0,20.0,100.0,100.0,20.0,100.0,100.0,20.0,Monitoramento Aprovado
16,2017-03-01,Brasil,Supply,2.0,195.0,Ger Engenharia,GER ENGENHARIA III,14.0,V-B,Ceng Campo,...,100.0,100.0,20.0,100.0,100.0,20.0,100.0,100.0,20.0,Monitoramento Aprovado
17,2017-03-01,Brasil,Supply,2.0,195.0,Ger Engenharia,GER ENGENHARIA III,14.0,V-B,Ceng Campo,...,100.0,100.0,20.0,100.0,100.0,20.0,100.0,100.0,20.0,Monitoramento Aprovado


In [120]:
for col in df.columns:
    if (df[col].loc[6] != df[col].loc[8]):
        print(str(col) + ": " + str(df[col].loc[6]) + " , " +  str(df[col].loc[8]))

Codigo_KPI: 001CC0161 , 001SC0434
Areas_da_Diretoria: CENG/CDT , Campo
Nome_KPI: 2424.0 , 4580.0
Regra_Alcance_Parcial: 100%: LTI (area) = 0 e LTI Fab <= Bgt e TRI (area) <= Bgt 60%: LTI (area) = 0 e TRI Area < LY e TRI (fabrica) <= Bgt Se ocorrer acidente fatal tipico = zero. , Conforme Matriz de Metas VPO 2017 disponivel no SDCA+


In [126]:
df['Tipo_da_Meta'].unique()

array(['Mandatorio', 'Grupo Area', 'Generico', 'Menu',
       'Metas Individuales 1', 'Metas Individuales 2',
       'Metas Individuales 3', 'Metas Individuales 4',
       'Metas Individuales 5'], dtype=object)

In [122]:
agg_df = df.groupby(['Nome_Funcionario','Mes_Referencia','Categoria_KPI']).nunique()

In [125]:
agg_df[agg_df['%_Pontos_Mes'].isna()
  & agg_df['%_Pontos_Acumulado'].isna()
  & agg_df['%_Pontos_Fim_Exer'].isna()].head(10)

,,,Mes_Referencia,Pais,Mundo,Regional/Area,Unidade,Grupo_Cargo,Cargo,Grade,Banda,Area,...,%_Ating_Mes,%_Pontos_Mes,%_Acum_Mes,%_Ating_Acumulado,%_Pontos_Acumulado,%_Acum_Acumulado,%_Ating_Fim_Exer,%_Pontos_Fim_Exer,%_Acum_Fim_Exer,Status_Meta
Nome_Funcionario,Mes_Referencia,Categoria_KPI,,,,,,,,,,,,,,,,,,,,,


In [131]:
df.loc[230478]

Mes_Referencia            2017-06-01 00:00:00
Pais                                 Barbados
Mundo                                  Supply
Regional/Area                               4
Unidade                                    16
Grupo_Cargo                               NaN
Cargo                                     NaN
Grade                                     NaN
Banda                                     NaN
Area                                      NaN
Nome_Funcionario                         6910
Nome_Gestor                               438
Codigo_KPI                                NaN
Diretoria                                 NaN
Areas_da_Diretoria                        NaN
Funcao                                    NaN
Tipo_da_Meta             Metas Individuales 1
Categoria_KPI                             NaN
Nome_KPI                                    0
Peso_KPI                                    0
Prazo                                     NaN
Regra_Alcance_Parcial             

In [135]:
df[(df['Nome_Funcionario']==6910)].groupby('Mes_Referencia').nunique()

,Mes_Referencia,Pais,Mundo,Regional/Area,Unidade,Grupo_Cargo,Cargo,Grade,Banda,Area,...,%_Ating_Mes,%_Pontos_Mes,%_Acum_Mes,%_Ating_Acumulado,%_Pontos_Acumulado,%_Acum_Acumulado,%_Ating_Fim_Exer,%_Pontos_Fim_Exer,%_Acum_Fim_Exer,Status_Meta
230477,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,80.0,16.0,NaN,100.0,20.0,NaN
230478,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN
230479,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230480,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN
230481,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,80.0,16.0,NaN,100.0,20.0,NaN
230482,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN
230483,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,100.0,20.0,NaN
230484,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN
230485,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230486,2017-06-01,Barbados,Supply,4.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN
